# 模式识别实验二：感知器算法

In [1]:
#导入库文件
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [2]:
#加载数据并显示
iris=datasets.load_iris()

#data对应了样本的4个特征，150行4列
[length,width] = iris.data.shape
print('iris dataset length:%d, width: %d' % (length,width)) 

#target对应样本的标签，150行1列
long = iris.target.shape
print('iris.target long : %d ' % long)

#为数据集整体添加偏置项
dataset = np.hstack((iris.data,np.ones((length,1))))
print('iris dataset with bias size: ',dataset.shape)

iris dataset length:150, width: 4
iris.target long : 150 
iris dataset with bias size:  (150, 5)


In [3]:
#数据集按照标签分开，用于两两分类训练权值
dataset1 = dataset[0:50,:]
labels1 = iris.target[0:50]

dataset2 = dataset[50:100,:]
labels2 = iris.target[50:100]

dataset3 = dataset[100:150,:]
labels3 = iris.target[100:150]

In [4]:
#检查样本是否都被正确分类
def check_fit(dataset,labels,alpha):
    value = dataset @ alpha
    value[value > 0] = labels[0]
    value[value < 0] = labels[-1]
    errors = abs(value - labels)
    error = np.mean(errors)
    return error

In [5]:
#定义权值更新函数 两两分类
#iteration 定义最大训练次数 定位2000
def train_function(dataset1,labels1,dataset2,labels2,iteration):
    #合并数据集dataset1 dataset2 
    dataset = np.vstack((dataset1,dataset2))
    labels = np.hstack((labels1,labels2))
    #权值初始化为（-0.5，0.5）之间的随机数
    alpha = np.random.random((5))- 0.5
    [length,width] = dataset.shape
    #取x_label为最后一个label 用于数据变号
    x_label = labels[-1]
    for i in range(iteration):
        indice = np.random.randint(0,length)
        data = dataset[indice]
        #如果为第二类样本中的数据，data取负
        if labels[indice] == x_label:
            data = -data
        value = data @ alpha
        if value <= 0:
            #step = abs(value) / sum(data * data)
            step = 1 
            alpha = alpha + step * data
        else:
            error = check_fit(dataset,labels,alpha)
            #控制训练集的误差在0.02以内
            if error > 0.02:
                continue
            else:
                print('train process finished')
                break        
    print('iteration: ',i)
    print('alpha: ', alpha)   
    return alpha

In [65]:
def predict_function(data,alpha12,alpha13,alpha23):
    value = np.zeros((3,))
    value[0] = data @ alpha12
    value[1] = data @ alpha13
    value[2] = data @ alpha23
    # value12 and vaule13 > 0
    if value[0] > 0 and value[1] > 0:
        predict = 0
    # value12<0 and value23 > 0
    elif value[0] < 0 and value[2] > 0:
        predict = 1
    # value13 and value 23 < 0
    elif value[1] < 0 and value[2] < 0:
        predict = 2
    #如果为IR区域，定为第4类
    else:
        predict = 3
    return predict

In [77]:
#传入分类结果和标签用于计算分类错误率
def error_ratio(result,labels):
    error = abs(result - labels)
    error[error > 0] = 1
    error_ratio = np.mean(error)
    print('error_ratio = ',error_ratio)

In [67]:
#将数据集分为训练集和测试集
X_train1,X_test1,y_train1,y_test1 = train_test_split(dataset1, labels1, test_size = 0.3, random_state = 0)
X_train2,X_test2,y_train2,y_test2 = train_test_split(dataset2, labels2, test_size = 0.3, random_state = 0)
X_train3,X_test3,y_train3,y_test3 = train_test_split(dataset3, labels3, test_size = 0.3, random_state = 0)

In [81]:
#训练得到三个权值向量
alpha12 = train_function(X_train1,y_train1,X_train2,y_train2,1000)
alpha13 = train_function(X_train1,y_train1,X_train3,y_train3,1000)
alpha23 = train_function(X_train2,y_train2,X_train3,y_train3,1000)

train process finished
iteration:  25
alpha:  [ 1.09788663  6.28964454 -8.8571606  -2.98949689  1.18888483]
train process finished
iteration:  16
alpha:  [  3.01911417   6.99082169 -10.19139253  -6.04451416   1.64422894]
train process finished
iteration:  27
alpha:  [ 2.52452252  1.59116369 -3.65704625 -2.02606035  0.87933523]


In [86]:
predict = np.zeros((length))
for i in range(length):
    predict[i] = predict_function(predict_dataset[i,:],alpha12,alpha13,alpha23)
#errors = abs(predict - predict_labels)
#print(errors)
error_ratio(predict,predict_labels)

error_ratio =  0.06666666666666667
